# Demo: Callers

A `Caller` is the basic structure that wraps all logic required for LLM call-and-response.

A `Caller` associates a `Prompt` with a specific LLM client and call parameters (assumes OpenAI-compatibility through a framework like `aisuite`).
This allows every Caller instance to use a different model and/or parameters, and sets expectations for the Caller instance.
Whereas `Prompts` validate _inputs_ to the template and `Handlers` validate the LLM responses, `Callers` make it all happen.

Additionally, `Callers` can be used as functions/tools in tool-calling workflows by leveraging `Caller.signature()` which provides the inputs the `Caller.prompt` requires as a JSON schema.
Since a `Caller` has a specific client and model assigned, this effectively allows us to use Callers to route to specific models for specific use cases.
Since Callers can behave as functions themselves, we enable complex workflows where Callers can call Callers (ad infinitum ad nauseum).

Simple factory functions create Callers where the use case is defined by their handlers:

- `ChatCaller`: a simple Caller implementation designed for chat messages without response validation.
- `RegexCaller`: uses regex for response validation.
- `StructuredCaller`:  is intended for structured responses, and uses Pydantic for response validation.
- `ToolCaller`: a configuration for tool-use; can optionally invoke the tool based on arguments in the LLM's response and return the function results.

In [ ]:
import json
import logging
import os
import re
import textwrap
from typing import cast

import json_repair
from pydantic import BaseModel, Field, ValidationError as PydanticValidationError, create_model

import aisuite
import openai

from yaaal.core.caller import Caller, create_chat_caller, create_structured_caller, create_tool_caller
from yaaal.core.handler import CompositeHandler, ResponseHandler, ToolHandler
from yaaal.core.prompt import (
    JinjaMessageTemplate,
    PassthroughMessageTemplate,
    Prompt,
    StaticMessageTemplate,
    StringMessageTemplate,
)
from yaaal.core.validator import PassthroughValidator, PydanticValidator, RegexValidator, ToolValidator
from yaaal.types.base import JSON
from yaaal.types.core import Conversation, Message
from yaaal.utilities import basic_log_config, format_json

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
basic_log_config()
logging.getLogger("yaaal").setLevel(logging.DEBUG)
logger = logging.getLogger(__name__).setLevel(logging.DEBUG)

## Quick Start

In [ ]:
# all Callers require a client and a model to call.
# `yaaal` is built around OpenAI-compatible APIs primarily provided by `aisuite`
client = aisuite.Client(
    provider_configs={
        "openai": {"api_key": os.environ["YAAAL_OPENAI_API_KEY"]},
        "anthropic": {"api_key": os.environ["YAAAL_ANTHROPIC_API_KEY"]},
        # ...
    }
)
# `aisuite` specifies models in "provider:model" format
model = "openai:gpt-4o-mini"

In [ ]:
# A `ChatCaller`
caller = create_chat_caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="chat",
        description="A simple chat",
        system_template=StaticMessageTemplate(role="system", template="You are a helpful assistant"),
        user_template=PassthroughMessageTemplate(),
    ),
)

2025-02-09 15:26:33,068 - DEBUG    - yaaal.core.caller - request_params:109 - All API requests for Caller will use params : {'temperature': 0.7}


In [ ]:
# callers can still render conversations through their prompt
caller.prompt.render(user_vars={"content": "Who is Harry Potter?"})

In [ ]:
# callers are called as functions to get the response from the LLM
response = caller(system_vars=None, user_vars={"content": "Who is Harry Potter?"})

print(textwrap.fill(response, replace_whitespace=False))

Harry Potter is a fictional character and the protagonist of the
"Harry Potter" series of novels written by British author J.K.
Rowling. The series follows Harry's journey from a young boy living
with his neglectful relatives, the Dursleys, to becoming a student at
Hogwarts School of Witchcraft and Wizardry. He discovers that he is a
wizard and learns about his past, including the fact that he is famous
in the wizarding world for surviving an attack by the dark wizard Lord
Voldemort when he was just a baby.

The series consists of seven
books: 

1. Harry Potter and the Sorcerer's Stone (also known as Harry
Potter and the Philosopher's Stone)
2. Harry Potter and the Chamber of
Secrets
3. Harry Potter and the Prisoner of Azkaban
4. Harry Potter
and the Goblet of Fire
5. Harry Potter and the Order of the Phoenix
6.
Harry Potter and the Half-Blood Prince
7. Harry Potter and the Deathly
Hallows

Throughout the series, Harry is joined by his friends
Hermione Granger and Ron Weasley as they f

In [ ]:
# Caller `schema` attribute is based on their Prompt signature
print(format_json(caller.schema))

In [ ]:
# A `RegexCaller` validates the response with a regex pattern
pattern = re.compile(r"\b[A-E]\b(?!.*\b[A-E]\b)")

template_str = """
"The following are multiple choice questions (with answers) about Star Wars.

What is the model designation of an X-Wing?
A. T-65B
B. BTL-A4
C. RZ-1
D. A/SF-01
Answer: A

{{question}}
Answer:
""".strip()


class MCQAQuestion(BaseModel):
    question: str = Field(description="The multiple choice question")


regex_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="chat",
        description="Multiple-choice question answering",
        system_template=JinjaMessageTemplate(role="system", template=template_str, template_vars_model=MCQAQuestion),
    ),
    handler=ResponseHandler(validator=RegexValidator(pattern=pattern)),
)

2025-02-09 15:27:40,785 - DEBUG    - yaaal.core.caller - request_params:109 - All API requests for Caller will use params : {'temperature': 0.7}


In [ ]:
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = regex_caller(system_vars={"question": question}, user_vars=None)

if response == "E":
    print("Success! 🎉")
# print(textwrap.fill(response, replace_whitespace=False))

Success! 🎉


In [ ]:
# A `StructuredCaller` validates the response with a Pydantic model, and is good for structure data extraction
class Person(BaseModel, extra="ignore"):
    name: str
    age: int
    favorite_color: str


# Use an fstring to create a jinja prompt --
# The fstring allows us to substitute in the Person schema.
# Because we're using fstrings, we have to double the `{}`
# so python understands they do not indicate an fstring substitution.
template_str = f"""
Identify facts about a person as they introduce themselves.

Respond in a format that matches the following schema:

<schema>
{Person.model_json_schema()}
</schema>

<introduction>
{{{{introduction}}}}
</introduction>
""".strip()


class PersonIntroduction(BaseModel):
    introduction: str


structured_caller = create_structured_caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="person details",
        description="Identify details about a person",
        system_template=JinjaMessageTemplate(
            role="system",
            template=template_str,
            template_vars_model=PersonIntroduction,
        ),
    ),
    response_model=Person,
)

2025-02-09 15:27:54,397 - DEBUG    - yaaal.core.caller - request_params:109 - All API requests for Caller will use params : {'temperature': 0.7}


In [ ]:
introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

response = structured_caller(system_vars={"introduction": introduction}, user_vars=None)

print(type(response))
print(format_json(response.model_dump()))

<class '__main__.Person'>
{
  "name": "Bob",
  "age": 42,
  "favorite_color": "blue",
}


Callers using Pydantic Handlers still return an AssistantMessage; it was validated internally before returning to the user.

This means we still have to re-validate if we want the response as a Pydantic model.

In [ ]:
# A `ToolCaller` can choose to call tools or respond like a normal LLM.

template_str = """Use the best tool for the task.""".strip()

tool_caller = create_tool_caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="tool use",
        description="Determine which tool to use",
        system_template=StaticMessageTemplate(role="system", template=template_str),
        user_template=PassthroughMessageTemplate(),
    ),
    toolbox=[regex_caller, structured_caller],  # we can use other callers as tools!
    auto_invoke=True,  # we should actually make the recommended tool call
)

2025-02-09 15:29:07,947 - DEBUG    - yaaal.core.caller - request_params:109 - All API requests for Caller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'chat', 'strict': True, 'parameters': {'$defs': {'MCQAQuestion': {'properties': {'question': {'description': 'The multiple choice question', 'title': 'Question', 'type': 'string'}}, 'required': ['question'], 'title': 'MCQAQuestion', 'type': 'object', 'additionalProperties': False}}, 'description': 'Multiple-choice question answering', 'properties': {'system_vars': {'$ref': '#/$defs/MCQAQuestion'}}, 'required': ['system_vars'], 'title': 'chat', 'type': 'object', 'additionalProperties': False}, 'description': 'Multiple-choice question answering'}}, {'type': 'function', 'function': {'name': 'person_details', 'strict': True, 'parameters': {'$defs': {'PersonIntroduction': {'properties': {'introduction': {'title': 'Introduction', 'type': 'string'}}, 'required': ['introduction'], 'title': 'PersonInt

In [ ]:
# the tool_caller will automatically add the tools to the request parameters
print(format_json(tool_caller.request_params))

In [ ]:
# this should call the person schema tool

introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

response = tool_caller(
    system_vars=None,
    user_vars={"content": introduction},
)

print(type(response))
print(format_json(response.model_dump()))

2025-02-09 15:31:29,813 - DEBUG    - yaaal.core.handler - process:115 - Invoking person_details


<class '__main__.Person'>
{
  "name": "Bob",
  "age": 42,
  "favorite_color": "blue",
}


In [ ]:
# this should call the Star Wars QA tool
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = tool_caller(
    system_vars=None,
    user_vars={"content": question},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

2025-02-09 15:31:50,601 - DEBUG    - yaaal.core.handler - process:115 - Invoking chat


<class 'str'>
"E"


In [ ]:
# this will error because tool_caller *only* calls tools
response = tool_caller(
    system_vars=None,
    user_vars={"content": "Hello world!"},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

2025-02-07 18:41:02,550 - DEBUG    - yaaal.core.caller - _handle_with_repair:151 - Repair 0 after error handling response Expected tool call but received none


ValidationError: No repair instructions available

A `tool_caller` as created above *only* calls tools.
To make a `Caller` that is also able to use standard chat completions, use a `CompositeHandler`

In [ ]:
all_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="tool use",
        description="Determine which tool to use",
        system_template=StaticMessageTemplate(role="system", template=template_str),
        user_template=PassthroughMessageTemplate(),
    ),
    handler=CompositeHandler(
        content_handler=ResponseHandler(PassthroughValidator()),
        tool_handler=ToolHandler(
            validator=ToolValidator(
                toolbox=[regex_caller, structured_caller],  # we can use other callers as tools!
            ),
            auto_invoke=True,  # we should actually make the recommended tool call
        ),
    ),
)

2025-02-09 15:32:10,127 - DEBUG    - yaaal.core.caller - request_params:109 - All API requests for Caller will use params : {'temperature': 0.7}


In [ ]:
response = all_caller(
    system_vars=None,
    user_vars={"content": "Hello world!"},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

<class 'str'>
"Hello! How can I assist you today?"


In [ ]:
# this should call the Star Wars QA tool
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = all_caller(
    system_vars=None,
    user_vars={"content": question},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

<class 'str'>
"E. All of the above"
